# LSTM による USD/JPY 対数収益予測（5分足・固定パラメータ）

PSO なし。固定パラメータで学習：
- **目的変数**: TARGET_HORIZON 本先の対数収益 $y_t = \ln(P_{t+h}/P_t)$（何本先かは設定で指定）
- **パラメータ**: units=208, epochs=182, layer=1

In [ ]:
from pso_lstm_common import (
    BATCH_SIZE,
    build_lstm_model,
    compute_metrics,
    create_sequences,
    inverse_scale_target,
    load_csv,
    preprocess_5m_pipeline,
    remove_high_corr_features,
    resample_ohlcv,
    sample_weights_from_scaled_target,
    scale_ewma_train_val_test,
    scale_target_ewma,
    train_val_test_split,
)

In [ ]:
import warnings
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import mixed_precision

import os

warnings.filterwarnings("ignore")
np.random.seed(42)
tf.random.set_seed(42)

policy = mixed_precision.Policy("mixed_float16")
mixed_precision.set_global_policy(policy)

physical_gpus = tf.config.list_physical_devices("GPU")
print(tf.config.list_physical_devices("GPU"))
if physical_gpus:
    for gpu in physical_gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"GPU利用: {len(physical_gpus)}台")
    if len(physical_gpus) > 1:
        strategy = tf.distribute.MirroredStrategy()
        print(
            f"複数GPUモード: MirroredStrategy で {strategy.num_replicas_in_sync} 台を使用"
        )
    else:
        strategy = None
else:
    print("GPUなし: CPUで実行します")
    strategy = None

In [ ]:
# ===== 設定 =====
DATA_PATH = "data/merged-usdjpy-base-2024-03-01-2025-10-31-5m.csv"
TARGET_HORIZON = 6
LOOKBACK = 1000
TRAIN_RATIO = 0.8
VAL_RATIO = 0.2
CORR_THRESHOLD = 0.95
TARGET_COL = "target_log_return"

# 固定パラメータ（PSO なし）
units = 200
epochs = 200
n_layers = 1

## 1. データ読込・前処理

In [ ]:
df_raw = load_csv(DATA_PATH)
df_raw = df_raw.head(20000)
df_raw = resample_ohlcv(df_raw, minutes=5)
df = preprocess_5m_pipeline(df_raw, steps_ahead=TARGET_HORIZON)
print(df.shape)
df.head(2)
# csvで保存
df.to_csv("data/merged-usdjpy-base-2024-03-01-2025-10-31-5m-processed.csv", index=False)

In [ ]:
exclude_from_features = {
    "target_log_return", "open", "high", "low", "close", "volume", "vwap",
    "EURUSD_close", "EURJPY_close",
}
feature_cols = [c for c in df.columns if c not in exclude_from_features]
df_work = df[feature_cols + [TARGET_COL]].copy()

# 末尾から target が NaN になっている部分を全て削除する
while df_work[TARGET_COL].isna().iloc[-1]:
    df_work = df_work.iloc[:-1]
# 前の値（前方）で埋める
df_work = df_work.ffill()
# NaNが残っているかチェック
if df_work.isnull().values.any():
    warnings.warn("前方補完してもNaNが残っています。データ先頭付近などにNaNがある可能性があります。")

df_work = df_work.dropna()
print("dropna 後:", df_work.shape)

In [ ]:
df_work, dropped = remove_high_corr_features(
    df_work, target_col=TARGET_COL, threshold=CORR_THRESHOLD
)
feature_cols = [c for c in df_work.columns if c != TARGET_COL]

print("削除した列:", dropped)
print("説明変数数:", len(feature_cols))

## 2. シーケンス作成・分割・スケーリング

In [ ]:
features = df_work[feature_cols].values
target = df_work[TARGET_COL].values
close_series = df_raw.reindex(df_work.index).ffill()["close"]

X, y, last_close = create_sequences(
    features, target, lookback=LOOKBACK, close_series=close_series
)
print("X:", X.shape, "y:", y.shape, "last_close:", last_close.shape)

In [ ]:
CACHE_PATH = "lstm_5m_fixed_cache.npz"

if os.path.exists(CACHE_PATH):
    data = np.load(CACHE_PATH, allow_pickle=True)
    X_train = data["X_train"]
    y_train = data["y_train"]
    X_val = data["X_val"]
    y_val = data["y_val"]
    X_test = data["X_test"]
    y_test = data["y_test"]
    lc_train = data["lc_train"]
    lc_val = data["lc_val"]
    lc_test = data["lc_test"]
    y_train_s = data["y_train_s"]
    y_val_s = data["y_val_s"]
    y_test_s = data["y_test_s"]
    mu_train = data["mu_train"]
    mu_val = data["mu_val"]
    mu_test = data["mu_test"]
    sigma_train = data["sigma_train"]
    sigma_val = data["sigma_val"]
    sigma_test = data["sigma_test"]
    sample_weight_train = data["sample_weight_train"]
    print("キャッシュから読み込み:", CACHE_PATH)
else:
    (
        X_train, y_train, X_val, y_val, X_test, y_test,
        lc_train, lc_val, lc_test,
    ) = train_val_test_split(X, y, train_ratio=TRAIN_RATIO, val_ratio=VAL_RATIO, last_close=last_close)
    print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

    # 目的変数を EWMA で標準化（設計書: Y_{t-h}～Y_{t-h-L+1} の窓で μ・σ、±3σ クリップ）
    target_full = df_work[TARGET_COL].values
    y_scaled, mu_arr, sigma_arr = scale_target_ewma(
        target_full, lookback=LOOKBACK, target_horizon=TARGET_HORIZON
    )
    n = len(X)
    train_end = int(n * TRAIN_RATIO)
    val_end = int(train_end * (1 - VAL_RATIO))
    y_train_s = y_scaled[:val_end]
    y_val_s = y_scaled[val_end:train_end]
    y_test_s = y_scaled[train_end:]
    mu_train, mu_val, mu_test = mu_arr[:val_end], mu_arr[val_end:train_end], mu_arr[train_end:]
    sigma_train, sigma_val, sigma_test = sigma_arr[:val_end], sigma_arr[val_end:train_end], sigma_arr[train_end:]
    sample_weight_train = sample_weights_from_scaled_target(y_train_s)

    np.savez(
        CACHE_PATH,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        X_test=X_test,
        y_test=y_test,
        lc_train=lc_train,
        lc_val=lc_val,
        lc_test=lc_test,
        y_train_s=y_train_s,
        y_val_s=y_val_s,
        y_test_s=y_test_s,
        mu_train=mu_train,
        mu_val=mu_val,
        mu_test=mu_test,
        sigma_train=sigma_train,
        sigma_val=sigma_val,
        sigma_test=sigma_test,
        sample_weight_train=sample_weight_train,
    )
    print("キャッシュを保存:", CACHE_PATH)

In [ ]:
if os.path.exists(CACHE_PATH):
    data = np.load(CACHE_PATH, allow_pickle=True)
    # すでにスケーリング済みデータが含まれていればそれを利用
    if all(k in data.files for k in ["X_train_s", "X_val_s", "X_test_s", "ewma_scaler"]):
        X_train_s = data["X_train_s"]
        X_val_s = data["X_val_s"]
        X_test_s = data["X_test_s"]
        # ewma_scaler はオブジェクトとして保存している想定
        ewma_scaler = data["ewma_scaler"].item()
        print("スケーリング結果をキャッシュから読み込み:", CACHE_PATH)
    else:
        # まだスケーリング結果がキャッシュにない場合は計算して追加で保存
        X_train_s, X_val_s, X_test_s, ewma_scaler = scale_ewma_train_val_test(
            X_train, X_val, X_test, feature_names=feature_cols
        )
        cache_dict = {k: data[k] for k in data.files}
        cache_dict.update(
            X_train_s=X_train_s,
            X_val_s=X_val_s,
            X_test_s=X_test_s,
            ewma_scaler=np.array(ewma_scaler, dtype=object),
        )
        np.savez(CACHE_PATH, **cache_dict)
        print("スケーリング結果をキャッシュに追加保存:", CACHE_PATH)
else:
    # セル10を飛ばしてここだけ実行した場合など、キャッシュが存在しないときは
    # 通常通りスケーリングのみ実行（キャッシュには保存しない）
    X_train_s, X_val_s, X_test_s, ewma_scaler = scale_ewma_train_val_test(
        X_train, X_val, X_test, feature_names=feature_cols
    )
    print("スケーリング完了（キャッシュファイル未作成のため未保存）")

print("スケーリング完了")

In [ ]:
import pandas as pd

# X_train_s は3次元 (サンプル数, lookbck, 変数数) なので2次元に変換してdescribeで確認
X_train_s_flat = X_train_s.reshape(-1, X_train_s.shape[-1])
print("X_train_s describe:")
print(pd.DataFrame(X_train_s_flat, columns=feature_cols).describe())

print("y_train describe:")
print(pd.Series(y_train_s.ravel()).describe())

## 3. モデル学習・テスト評価

In [ ]:
input_shape = (LOOKBACK, len(feature_cols))
tf.keras.backend.clear_session()
if strategy is not None:
    with strategy.scope():
        final_model = build_lstm_model(input_shape, n_layers, units)
else:
    final_model = build_lstm_model(input_shape, n_layers, units)

# 最終学習用コールバック: EarlyStopping と ModelCheckpoint
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=1000,
    restore_best_weights=True,
)
ckpt_path = "best_model_pso_lstm_5m.keras"
model_checkpoint = keras.callbacks.ModelCheckpoint(
    ckpt_path,
    monitor="val_loss",
    # save_best_only=True,
)
csv_logger = keras.callbacks.CSVLogger("final_log_pso_lstm_5m.csv")


final_model.fit(
    X_train_s,
    y_train_s,
    sample_weight=sample_weight_train,
    validation_data=(X_val_s, y_val_s),
    epochs=epochs,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, model_checkpoint, csv_logger],
    verbose=0,
)

In [ ]:
# 予測は標準化空間で出力されるため、μ_t, σ_t で元スケールに戻してから評価する
y_pred_scaled = final_model.predict(X_test_s, verbose=0).ravel()
y_pred = inverse_scale_target(y_pred_scaled, mu_test, sigma_test)
y_test_orig = inverse_scale_target(y_test_s, mu_test, sigma_test)
rmse, mae, mape, r2 = compute_metrics(y_test_orig, y_pred)
print(f"テスト RMSE={rmse:.6f}, MAE={mae:.6f}, MAPE={mape:.4f}%, R2={r2:.4f}")

In [ ]:
# オプション: モデルとテスト・訓練結果を保存（可視化ノートブックで利用）
final_model.save("best_model_pso_lstm_5m.keras")
np.savez("test_result.npz", y_test=y_test_orig, y_pred=y_pred, lc_test=lc_test)

# 訓練データの予測を計算して保存（元スケール）
y_train_pred_scaled = final_model.predict(X_train_s, verbose=0).ravel()
y_train_pred = inverse_scale_target(y_train_pred_scaled, mu_train, sigma_train)
y_train_orig = inverse_scale_target(y_train_s, mu_train, sigma_train)
np.savez("train_result.npz", y_train=y_train_orig, y_pred=y_train_pred, lc_train=lc_train)

In [ ]:
pred_close = lc_test * np.exp(y_pred)
actual_close = lc_test * np.exp(y_test_orig)
print("予測終値と実測終値のサンプル（先頭5件）:")
print("actual:", actual_close[:5])
print("pred:  ", pred_close[:5])

In [ ]:
print("std(y_train_orig)   =", np.std(y_train_orig))
print("std(y_train_pred) =", np.std(y_train_pred))
print("ratio (実測/予測) =", np.std(y_train_orig) / np.std(y_train_pred))
